In [1]:
import torch
import dgl

In [11]:
graph_name = "ogbn-arxiv"
TARGET_DIR = "/home/ubuntu/data/" + graph_name
cache_size_in_GB = "1"
with open(TARGET_DIR + "/meta.txt",'r') as fp:
    lines = fp.readlines()
meta = {}
for line in lines:
    k, v = line.split("=")
    meta[k] = int(v)

with open(TARGET_DIR + "/partition_offsets.txt", 'r') as fp:
    lines = fp.readlines()
partition_offsets = lines[0].split(",")    
partition_offsets = [int(i) for i in partition_offsets]
print(meta)
print(partition_offsets)

{'num_nodes': 169343, 'num_edges': 1166243, 'csum_features': 444010, 'feature_dim': 128, 'csum_labels': 3501834, 'csum_train': 8067583445, 'csum_test': 2444589568, 'csum_offsets': 110140692313, 'csum_edges': 81107407091, 'num_classes': 40, 'fbytes': 32}
[0, 43580, 87187, 129587, 169343]


In [12]:
from utils import utils
graphname = "ogbn-arxiv"
g = utils.get_process_graph(graphname, -1, 4)

TODO: Refactor this not all function calls require the graph to be read
169343 1166243 40 CLASS
tensor([0, 1, 2, 2, 2, 2, 3, 3, 3, 3])
Using 32
Scipy created
DG Graph Created
All data created
Total time to read the graph 0.08505702018737793


In [13]:
train_nodes = g[0].ndata['train_mask'].nonzero().flatten()
valid_nodes = g[0].ndata['val_mask'].nonzero().flatten()

In [18]:
for gpu_id in range(4):
    num_train = torch.sum((train_nodes >= partition_offsets[gpu_id]) & \
                             (train_nodes < partition_offsets[gpu_id + 1]))
    num_valid = torch.sum((valid_nodes >= partition_offsets[gpu_id]) & \
                             (valid_nodes < partition_offsets[gpu_id + 1]))
    print(f"partition:{gpu_id},  training nodes {num_train}, validation nodes {num_valid}")

partition:0,  training nodes 23418, validation nodes 7884
partition:1,  training nodes 23418, validation nodes 7664
partition:2,  training nodes 23418, validation nodes 6860
partition:3,  training nodes 20687, validation nodes 7391
